## 7. 프로젝트: 멋진 작사가 만들기

### Step 1. 데이터 다운로드

In [ ]:
#$ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
#$ unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics  #lyrics 폴더에 압축풀기

### Step 2. 데이터 읽어오기    

glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요. glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 할게요!

In [7]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Another one', 'We The Best music', "DJ Khaled I don't know if you could take it"]


In [8]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os
 
# 파일을 읽기모드로 열어 봅니다.
file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/Kanye_West.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()   # 텍스트를 라인 단위로 끊어서 list 형태로 읽어옵니다.

print(raw_corpus[:9])    # 앞에서부터 10라인만 화면에 출력해 볼까요?

['\ufeffEighteen years eighteen years', 'She got one of your kids got you for eighteen years', 'I know somebody paying child support for one of his kids', 'His baby mama car and crib is bigger than his', 'You will see him on TV any given Sunday', 'Win the Super Bowl and drive off in a Hyundai', 'She was supposed to buy your shorty Tyco with your money', 'She went to the doctor got lipo with your money', 'She walking around looking like Michael with your money']


#### 데이터를 살펴보니 노래 가사인듯 보이네요. 

In [9]:
for idx, sentence in enumerate(raw_corpus):

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

﻿Eighteen years eighteen years
She got one of your kids got you for eighteen years
I know somebody paying child support for one of his kids
His baby mama car and crib is bigger than his
You will see him on TV any given Sunday
Win the Super Bowl and drive off in a Hyundai
She was supposed to buy your shorty Tyco with your money
She went to the doctor got lipo with your money
She walking around looking like Michael with your money
Shouldve got that insured Geico for your money


#### 우리가 원하는 문장이 성공적으로 출력되었습니다

### Step 3. 데이터 정제   


앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!   



preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.   



추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래가사 작사하기에 어울리지 않을수도 있겠죠.   
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하기를 권합니다.   

In [26]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


#### 문장부호를 삭제하고, 대문자를 소문자로 변환하고, 특수문자를 모드 제거하는 지저분한 문장을 깨끗하게 정제해 주는 함수가 완성되었습니다.

In [55]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue 
    # if len(corpus) >= 15: del    
    corpus.append(preprocess_sentence(sentence))
          
corpus[:10]

['<start> eighteen years eighteen years <end>',
 '<start> she got one of your kids got you for eighteen years <end>',
 '<start> i know somebody paying child support for one of his kids <end>',
 '<start> his baby mama car and crib is bigger than his <end>',
 '<start> you will see him on tv any given sunday <end>',
 '<start> win the super bowl and drive off in a hyundai <end>',
 '<start> she was supposed to buy your shorty tyco with your money <end>',
 '<start> she went to the doctor got lipo with your money <end>',
 '<start> she walking around looking like michael with your money <end>',
 '<start> shouldve got that insured geico for your money <end>']

#### 소스문장과 타켓문장을 생성하기 위한 단계로 정제함수를 통해  완벽하게 데이터를 준비 했습니다. 

### Step 4. 평가 데이터셋 분리   


훈련 데이터와 평가 데이터를 분리하세요!   



tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!   

In [ ]:
#enc_train, enc_val, dec_train, dec_val = <코드 작성>

In [66]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=16, truncating='post',padding='post')  
                                    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 931 243 ...   0   0   0]
 [  2  25  26 ...   0   0   0]
 [  2   5  29 ...   0   0   0]
 ...
 [  2  24   6 ...   0   0   0]
 [  2  10 217 ...  13   3   0]
 [  2  28  12 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f68d027f490>


####  토큰화를 진행 하고 토큰개수 15개가 넘는 단어들을 제거하고 싶었지만 제 짧은 실력으로는 아무리 if문을 써보고 이것 저것을 다 해봐도 해결할수가 없었지만  방법이 없어서 그냥 패딩이 과도하게 들어가더라도 진행하려 했는데 막상 하다보니 신경이 쓰여서 한참 구글링을 해보니 패딩을 주는 과정에서도 데이터를 잘라낼수 있는 방법이 있어서 한번 시도해 봤더니 보기좋게 삭제가 되었네요. 기분좋습니다~

In [67]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : the
5 : i
6 : you
7 : a
8 : and
9 : to
10 : my


#### 단어 사전이 아주 잘 구축되어 있는걸 확인할수 있습니다.

In [70]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2 931 243 931 243   3   0   0   0   0   0   0   0   0   0]
[931 243 931 243   3   0   0   0   0   0   0   0   0   0   0]


In [82]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

NameError: name 'enc_train' is not defined

#### 생성된 텐서를 소스와 타겟으로 분리하여 모델이 학습할 수 있게 하였습니다.

In [80]:
import numpy as np
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val =

[[  2 931 243 ...   0   0   0]
 [  2  25  26 ...   0   0   0]
 [  2   5  29 ...   0   0   0]
 ...
 [  2  24   6 ...   0   0   0]
 [  2  10 217 ...  13   3   0]
 [  2  28  12 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f684c7d2f90>


NameError: name 'data' is not defined

In [34]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

In [35]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [36]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 20, 12001), dtype=float32, numpy=
array([[[-1.13232825e-04, -1.79957497e-04,  1.87084021e-04, ...,
         -3.26394103e-04, -8.45029208e-05,  2.28053948e-04],
        [-1.96843423e-04, -3.26531037e-04,  2.97538354e-04, ...,
         -4.25766688e-04, -1.46763734e-04,  4.69094317e-04],
        [-2.44447147e-04, -4.12672758e-04,  3.81603517e-04, ...,
         -4.28438478e-04, -3.29473551e-04,  6.09909592e-04],
        ...,
        [-6.53210853e-04, -1.03181133e-04, -1.23922480e-03, ...,
         -3.71217640e-04, -1.88861985e-03, -2.89449294e-04],
        [-4.72459971e-04,  1.74342524e-04, -1.54902868e-03, ...,
         -4.23955062e-04, -1.80421211e-03, -2.86192371e-04],
        [-3.19634477e-04,  4.65171470e-04, -1.86414702e-03, ...,
         -4.53660439e-04, -1.66629604e-03, -2.68252013e-04]],

       [[-1.13232825e-04, -1.79957497e-04,  1.87084021e-04, ...,
         -3.26394103e-04, -8.45029208e-05,  2.28053948e-04],
        [-1.92641077e-04, -2.19940921e-06,  1

In [37]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


 여기까지 올바르게 진행했을 경우, 아래 실행 결과를 확인할 수 있습니다.   
 
 
 
print("Source Train:", enc_train.shape)   
print("Target Train:", dec_train.shape)   


out:   



Source Train: (124960, 14)   
Target Train: (124960, 14)   


만약 결과가 다르다면 천천히 과정을 다시 살펴 동일한 결과를 얻도록 하세요! 만약 학습데이터 갯수가 124960보다 크다면 위 Step 3.의 데이터 정제 과정을 다시한번 검토해 보시기를 권합니다. 

### Step 5. 인공지능 만들기   

모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)   



그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

In [ ]:
#Loss

#loss = tf.keras.losses.SparseCategoricalCrossentropy(
#    from_logits=True, reduction='none')


데이터가 커서 훈련하는 데 시간이 제법 걸릴 겁니다. 여유를 가지고 작업하시면 좋아요 :)   




generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)